<a href="https://colab.research.google.com/github/vvamsi91/IML_AS7/blob/main/IML_AS7_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import datetime


In [ ]:
torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)
np.random.seed(123)


In [ ]:
# Setting up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cpu')

In [ ]:
import torchvision.transforms as transforms
import torch.nn.functional as F

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())

100%|██████████| 170498071/170498071 [00:02<00:00, 80582378.27it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [ ]:


transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])
cifar10 = datasets.CIFAR10('./data', train=True, download=False, transform=transform)


In [ ]:
cifar10_val = datasets.CIFAR10('./data', train=False, download=False,
                               transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))]))


In [ ]:
train_loader = DataLoader(cifar10, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(cifar10_val, batch_size=32, shuffle=False, num_workers=2)
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
print(torch.__version__)

2.1.0+cu118


In [ ]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        if epoch == 1 or epoch % 2 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))

In [ ]:
class ResBlock(nn.Module):
    def __init__(self, n_chans):
        super(ResBlock, self).__init__()
        self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3, padding=1, bias=False)
        self.batch_norm = nn.BatchNorm2d(num_features=n_chans)
        nn.init.kaiming_normal_(self.conv.weight, nonlinearity='relu')
        nn.init.constant_(self.batch_norm.weight, 0.5)
        nn.init.zeros_(self.batch_norm.bias)

    def forward(self, x):
        return torch.relu(self.batch_norm(self.conv(x))) + x


In [ ]:
class NetResDeep(nn.Module):
    def __init__(self, n_chans1=32, n_blocks=10):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.resblocks = nn.Sequential(
            *(n_blocks * [ResBlock(n_chans=n_chans1)]))
        self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = self.resblocks(out)
        out = F.max_pool2d(out, 2)
        out = out.view(-1, 8 * 8 * self.n_chans1)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [ ]:
print(device)

cuda


In [ ]:
model = NetResDeep(n_chans1=32, n_blocks=10).to(device=device)
optimizer = optim.SGD(model.parameters(), lr=3e-3)
loss_fn = nn.CrossEntropyLoss()
training_loop(
    n_epochs = 300,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)


2023-12-12 23:34:56.526688 Epoch 1, Training loss 1.578044022647372
2023-12-12 23:35:27.492537 Epoch 2, Training loss 1.2774697199358973
2023-12-12 23:36:32.293571 Epoch 4, Training loss 1.0684302118597928
2023-12-12 23:37:34.856660 Epoch 6, Training loss 0.9435079231982192
2023-12-12 23:38:38.427915 Epoch 8, Training loss 0.8640125250671433
2023-12-12 23:39:41.109520 Epoch 10, Training loss 0.8033782051148052
2023-12-12 23:40:45.517788 Epoch 12, Training loss 0.7533268784378403
2023-12-12 23:41:48.510045 Epoch 14, Training loss 0.7069631824490358
2023-12-12 23:42:52.438472 Epoch 16, Training loss 0.670059420039695
2023-12-12 23:43:55.241447 Epoch 18, Training loss 0.6337188809862216
2023-12-12 23:44:57.774047 Epoch 20, Training loss 0.5957012096049308
2023-12-12 23:45:59.431344 Epoch 22, Training loss 0.5671416467874384
2023-12-12 23:47:00.945944 Epoch 24, Training loss 0.5380308904607023
2023-12-12 23:48:02.688664 Epoch 26, Training loss 0.5092968434924814
2023-12-12 23:49:03.341639 

In [ ]:
def validate(model, train_loader, val_loader):
    accdict = {}

    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0
        predictions, exp_labels = [], []

        with torch.no_grad():
            for imgs, labels in loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.shape[0]
                correct += (predicted == labels).sum().item()

                predictions.extend(predicted.cpu().numpy())
                exp_labels.extend(labels.cpu().numpy())

        accuracy = correct / total
        print(f"Accuracy {name}: {accuracy:.2f}")
        accdict[name] = accuracy

    return accdict, predictions, exp_labels

train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)


In [ ]:
accuracy, predictions, expected_labels = validate(model, train_loader, val_loader)

Accuracy train: 0.99
Accuracy val: 0.66
time: 22.6 s (started: 2023-12-13 02:08:43 +00:00)


In [ ]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix

precision, recall = precision_score(predictions, expected_labels, average='macro'), recall_score(predictions, expected_labels, average='macro')
cnf_matrix = confusion_matrix(predictions, expected_labels)


In [ ]:
print(classification_report(predictions, expected_labels, target_names=class_names))

              precision    recall  f1-score   support

    airplane       0.72      0.69      0.70      1052
  automobile       0.75      0.83      0.79       896
        bird       0.57      0.51      0.53      1116
         cat       0.48      0.46      0.47      1037
        deer       0.64      0.61      0.62      1047
         dog       0.52      0.57      0.54       923
        frog       0.68      0.78      0.73       873
       horse       0.71      0.73      0.72       975
        ship       0.77      0.79      0.78       968
       truck       0.81      0.72      0.76      1113

    accuracy                           0.66     10000
   macro avg       0.66      0.67      0.67     10000
weighted avg       0.66      0.66      0.66     10000

time: 52.5 ms (started: 2023-12-13 02:09:07 +00:00)


In [ ]:
class NetDropout(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv1_dropout = nn.Dropout2d(p=0.3)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3, padding=1)
        self.conv2_dropout = nn.Dropout2d(p=0.3)
        self.fc = nn.Sequential(
            nn.Linear(8 * 8 * n_chans1 // 2, 32),
            nn.Tanh(),
            nn.Linear(32, 10)
        )

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = self.conv1_dropout(out)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = self.conv2_dropout(out)
        return self.fc(out.view(-1, 8 * 8 * self.n_chans1 // 2))


In [ ]:
model_dropout = NetDropout(n_chans1=32).to(device=device)
optimizer_dropout = optim.SGD(model_dropout.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 300,
    optimizer = optimizer_dropout,
    model = model_dropout,
    loss_fn = loss_fn,
    train_loader = train_loader,
)


2023-12-13 02:09:23.387425 Epoch 1, Training loss 2.0211501491953956
2023-12-13 02:09:38.183454 Epoch 2, Training loss 1.7908894675771903
2023-12-13 02:10:08.669378 Epoch 4, Training loss 1.563987286194511
2023-12-13 02:10:38.959994 Epoch 6, Training loss 1.4745130110579683
2023-12-13 02:11:10.133252 Epoch 8, Training loss 1.4042231009134551
2023-12-13 02:11:40.428335 Epoch 10, Training loss 1.3402042797459361
2023-12-13 02:12:11.413832 Epoch 12, Training loss 1.2961306644370183
2023-12-13 02:12:41.722395 Epoch 14, Training loss 1.259879713015788
2023-12-13 02:13:13.251650 Epoch 16, Training loss 1.2310235088743517
2023-12-13 02:13:43.756423 Epoch 18, Training loss 1.2080232135932465
2023-12-13 02:14:15.067404 Epoch 20, Training loss 1.186558669028075
2023-12-13 02:14:45.552902 Epoch 22, Training loss 1.1730980440173917
2023-12-13 02:15:16.358477 Epoch 24, Training loss 1.1533615790364686
2023-12-13 02:15:46.880985 Epoch 26, Training loss 1.1397344309198276
2023-12-13 02:16:17.955112 E

In [ ]:
accuracy_dropout, predictions_dropout, expected_labels_dropout = validate(model_dropout, train_loader, val_loader)
from sklearn.metrics import precision_score, recall_score, confusion_matrix


precision_dropout, recall_dropout = precision_score(predictions_dropout, expected_labels_dropout, average='macro'), recall_score(predictions_dropout, expected_labels_dropout, average='macro')
cnf_matrix_dropout = confusion_matrix(predictions_dropout, expected_labels_dropout)


Accuracy train: 0.69
Accuracy val: 0.61
time: 16 s (started: 2023-12-13 03:26:25 +00:00)


In [ ]:
print(classification_report(predictions_dropout, expected_labels_dropout, target_names=class_names))


              precision    recall  f1-score   support

    airplane       0.71      0.62      0.66      1155
  automobile       0.73      0.73      0.73      1008
        bird       0.46      0.48      0.46       957
         cat       0.40      0.42      0.41       949
        deer       0.55      0.57      0.56       967
         dog       0.52      0.52      0.52      1003
        frog       0.65      0.68      0.67       955
       horse       0.67      0.66      0.66      1006
        ship       0.71      0.74      0.73       963
       truck       0.69      0.67      0.68      1037

    accuracy                           0.61     10000
   macro avg       0.61      0.61      0.61     10000
weighted avg       0.61      0.61      0.61     10000

time: 48.1 ms (started: 2023-12-13 03:26:42 +00:00)


In [ ]:
class NetDropout(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv1_dropout = nn.Dropout2d(p=0.3)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3, padding=1)
        self.conv2_dropout = nn.Dropout2d(p=0.3)
        self.fc = nn.Sequential(
            nn.Linear(8 * 8 * n_chans1 // 2, 32),
            nn.Tanh(),
            nn.Linear(32, 10)
        )

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = self.conv1_dropout(out)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = self.conv2_dropout(out)
        return self.fc(out.view(-1, 8 * 8 * self.n_chans1 // 2))


In [ ]:
model_dropout = NetDropout(n_chans1=32).to(device=device)
optimizer_dropout = optim.SGD(model_dropout.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 150,
    optimizer = optimizer_dropout,
    model = model_dropout,
    loss_fn = loss_fn,
    train_loader = train_loader,
)


2023-12-13 03:30:19.491158 Epoch 1, Training loss 1.995631134571017
2023-12-13 03:30:35.413092 Epoch 2, Training loss 1.7166142004835026
2023-12-13 03:31:04.297918 Epoch 4, Training loss 1.5339118175189514
2023-12-13 03:31:33.365952 Epoch 6, Training loss 1.4455803617492051
2023-12-13 03:32:02.049553 Epoch 8, Training loss 1.3710254606078653
2023-12-13 03:32:30.829585 Epoch 10, Training loss 1.3151189550719298
2023-12-13 03:32:59.980428 Epoch 12, Training loss 1.2685180420765791
2023-12-13 03:33:29.915862 Epoch 14, Training loss 1.2409061988448853
2023-12-13 03:33:59.286316 Epoch 16, Training loss 1.2171496484438171
2023-12-13 03:34:29.038043 Epoch 18, Training loss 1.1916198180154767
2023-12-13 03:34:58.821512 Epoch 20, Training loss 1.172063709982216
2023-12-13 03:35:28.510125 Epoch 22, Training loss 1.1587711659538777
2023-12-13 03:35:57.758815 Epoch 24, Training loss 1.1409421938154705
2023-12-13 03:36:27.770373 Epoch 26, Training loss 1.1297474248939767
2023-12-13 03:36:58.094917 

In [ ]:
accuracy_dropout, predictions_dropout, expected_labels_dropout = validate(model_dropout, train_loader, val_loader)


Accuracy train: 0.68
Accuracy val: 0.62


In [ ]:


precision_dropout, recall_dropout, cnf_matrix_dropout = precision_score(predictions_dropout, expected_labels_dropout, average='macro'), recall_score(predictions_dropout, expected_labels_dropout, average='macro'), confusion_matrix(predictions_dropout, expected_labels_dropout)


In [ ]:
print(classification_report(predictions_dropout, expected_labels_dropout, target_names=class_names))

              precision    recall  f1-score   support

    airplane       0.70      0.66      0.68      1053
  automobile       0.77      0.72      0.74      1063
        bird       0.48      0.52      0.50       923
         cat       0.44      0.45      0.44       988
        deer       0.54      0.54      0.54      1001
         dog       0.54      0.53      0.53      1030
        frog       0.70      0.70      0.70       991
       horse       0.63      0.67      0.65       939
        ship       0.76      0.72      0.74      1058
       truck       0.68      0.71      0.69       954

    accuracy                           0.62     10000
   macro avg       0.62      0.62      0.62     10000
weighted avg       0.63      0.62      0.62     10000



In [ ]:
class NetBatchNorm(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv1_batchnorm = nn.BatchNorm2d(num_features=n_chans1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3, padding=1)
        self.conv2_batchnorm = nn.BatchNorm2d(num_features=n_chans1 // 2)
        self.fc = nn.Sequential(
            nn.Linear(8 * 8 * n_chans1 // 2, 32),
            nn.Tanh(),
            nn.Linear(32, 10)
        )

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1_batchnorm(self.conv1(x))), 2)
        out = F.max_pool2d(torch.tanh(self.conv2_batchnorm(self.conv2(out))), 2)
        return self.fc(out.view(-1, 8 * 8 * self.n_chans1 // 2))


In [ ]:
model_batch_norm = NetBatchNorm(n_chans1=32).to(device=device)
optimizer_batch_norm = optim.SGD(model_batch_norm.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 100,
    optimizer = optimizer_batch_norm,
    model = model_batch_norm,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

2023-12-13 04:07:30.870113 Epoch 1, Training loss 1.807895163898273
2023-12-13 04:07:45.275596 Epoch 2, Training loss 1.482054467396358
2023-12-13 04:08:13.294459 Epoch 4, Training loss 1.2449691876426072
2023-12-13 04:08:41.692687 Epoch 6, Training loss 1.113484408087133
2023-12-13 04:09:10.446882 Epoch 8, Training loss 1.0256894990001493
2023-12-13 04:09:39.203947 Epoch 10, Training loss 0.9600044155059873
2023-12-13 04:10:07.580682 Epoch 12, Training loss 0.9097343064330118
2023-12-13 04:10:36.053742 Epoch 14, Training loss 0.8703683008775687
2023-12-13 04:11:04.069054 Epoch 16, Training loss 0.8380810820385624
2023-12-13 04:11:32.342050 Epoch 18, Training loss 0.8104771412242099
2023-12-13 04:12:00.744576 Epoch 20, Training loss 0.7861267812645344
2023-12-13 04:12:28.729499 Epoch 22, Training loss 0.7642831300835475
2023-12-13 04:12:57.045642 Epoch 24, Training loss 0.7443065306033625
2023-12-13 04:13:25.116664 Epoch 26, Training loss 0.7255052128411315
2023-12-13 04:13:53.047694 E